In [6]:
import logging
from clickhouse_driver import Client
from dotenv import load_dotenv
import os

# Загрузка перементов окружения
load_dotenv()

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("load_olapcube.log", encoding='utf-8'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger()

# Параметры подключения
clickhouse_host = os.getenv('CLICKHOUSE_HOST', '10.95.19.132')
clickhouse_user = os.getenv('CLICKHOUSE_USER', 'default')
clickhouse_password = os.getenv('CLICKHOUSE_PASSWORD', 'quie1ahpoo5Su0wohpaedae8keeph6bi')
database_name = os.getenv('CLICKHOUSE_DB', 'default')

# Создание клиента ClickHouse
client = Client(
    host=clickhouse_host,
    user=clickhouse_user,
    password=clickhouse_password,
    port=9000,
    secure=False,
    settings={'strings_encoding': 'utf-8'}
)

def calculate_aggregate_count():
    try:
        # Суммирование mi17_count и mi8t_count
        query_fleet = """
        SELECT 
            SUM(mi17_count) AS total_mi17_count, 
            SUM(mi8t_count) AS total_mi8t_count 
        FROM fleet
        """
        result_fleet = client.execute(query_fleet)
        total_mi17_count, total_mi8t_count = result_fleet[0]
        logger.info(f"Сумма mi17_count: {total_mi17_count}, сумма mi8t_count: {total_mi8t_count}")

        # Получение значений из Agregat
        query_agregat = """
        SELECT 
            items_per_ac, 
            NR_Mi17 
        FROM Agregat 
        LIMIT 1
        """
        result_agregat = client.execute(query_agregat)
        if not result_agregat:
            logger.error("Таблица Agregat пуста. Невозможно вычислить aggregate_count.")
            return None
        items_per_ac, NR_Mi17 = result_agregat[0]
        logger.info(f"items_per_ac: {items_per_ac}, NR_Mi17: {NR_Mi17}")

        if NR_Mi17 == 0:
            logger.error("NR_Mi17 равно 0. Деление невозможно.")
            return None

        # Получение monthly_flight_hours
        query_flight_hours = """
        SELECT 
            monthly_flight_hours 
        FROM flight_hours 
        LIMIT 1
        """
        result_flight_hours = client.execute(query_flight_hours)
        if not result_flight_hours:
            logger.error("Таблица flight_hours пуста. Невозможно получить monthly_flight_hours.")
            return None
        monthly_flight_hours = result_flight_hours[0][0]
        logger.info(f"monthly_flight_hours: {monthly_flight_hours}")

        # Расчет aggregate_count
        aggregate_count = (total_mi17_count + total_mi8t_count) * 11 * 1.5 * (items_per_ac * monthly_flight_hours) / NR_Mi17
        aggregate_count = round(aggregate_count)
        logger.info(f"Вычисленный aggregate_count: {aggregate_count}")
        return aggregate_count

    except Exception as e:
        logger.error(f"Ошибка при вычислении aggregate_count: {e}", exc_info=True)
        return None

def get_agregat_values():
    try:
        query = "SELECT MRR_Mi8T, MRR_Mi17, MRR_2_Mi8T, MRR_2_Mi17, NR_Mi8T, NR_Mi17, BR_Mi8T, BR_Mi17 FROM Agregat LIMIT 1"
        result = client.execute(query)
        if not result:
            logger.error("Таблица Agregat пуста. Невозможно получить значения.")
            return None
        MRR_Mi8T, MRR_Mi17, MRR_2_Mi8T, MRR_2_Mi17, NR_Mi8T, NR_Mi17, BR_Mi8T, BR_Mi17 = result[0]
        logger.info("Получены значения из таблицы Agregat.")
        return {
            'MRR_Mi8T': MRR_Mi8T,
            'MRR_Mi17': MRR_Mi17,
            'MRR_2_Mi8T': MRR_2_Mi8T,
            'MRR_2_Mi17': MRR_2_Mi17,
            'NR_Mi8T': NR_Mi8T,
            'NR_Mi17': NR_Mi17,
            'BR_Mi8T': BR_Mi8T,
            'BR_Mi17': BR_Mi17
        }
    except Exception as e:
        logger.error(f"Ошибка при получении значений из Agregat: {e}", exc_info=True)
        return None

def get_daily_flight_hours():
    try:
        query = "SELECT daily_flight_hours FROM flight_hours LIMIT 1"
        result = client.execute(query)
        if not result:
            logger.error("Таблица flight_hours пуста. Невозможно получить daily_flight_hours.")
            return None
        daily_flight_hours = result[0][0]
        logger.info(f"Получено daily_flight_hours: {daily_flight_hours}")
        return daily_flight_hours
    except Exception as e:
        logger.error(f"Ошибка при получении daily_flight_hours: {e}", exc_info=True)
        return None

def insert_into_olapcube_vnv_with_dates(aggregate_count, agregat_values, daily_flight_hours):
    try:
        if aggregate_count is None or agregat_values is None or daily_flight_hours is None:
            logger.error("Не все данные для вставки в OlapCube_VNV доступны.")
            return

        # Подготовка данных
        threshold = agregat_values['MRR_Mi8T'] if agregat_values['MRR_Mi8T'] and agregat_values['MRR_Mi8T'] != 0 else agregat_values['MRR_Mi17']
        oh = agregat_values['MRR_2_Mi8T'] if agregat_values['MRR_2_Mi8T'] and agregat_values['MRR_2_Mi8T'] != 0 else agregat_values['MRR_2_Mi17']
        ll = agregat_values['NR_Mi8T'] if agregat_values['NR_Mi8T'] and agregat_values['NR_Mi8T'] != 0 else agregat_values['NR_Mi17']
        BR = agregat_values['BR_Mi8T'] if agregat_values['BR_Mi8T'] and agregat_values['BR_Mi8T'] != 0 else agregat_values['BR_Mi17']

        # Получение всех уникальных дат
        query_dates = f"SELECT DISTINCT Dates FROM {database_name}.OlapCube_VNV ORDER BY Dates ASC"
        dates = client.execute(query_dates)
        if not dates:
            logger.error("Поле Dates в таблице OlapCube_VNV пусто.")
            return
        dates = [date[0] for date in dates]
        logger.info(f"Получено {len(dates)} уникальных дат.")

        # Генерация серийных номеров
        serial_numbers = [f"S{i + 1}" for i in range(aggregate_count)]
        logger.info(f"Сгенерировано {len(serial_numbers)} серийных номеров.")

        # Подготовка данных для вставки
        insert_query = f"""
        INSERT INTO {database_name}.OlapCube_VNV (
            serialno, Dates, threshold, oh, ll, BR, daily_flight_hours, Status, sne, ppr
        ) VALUES
        """
        data_to_insert = []
        for serial in serial_numbers:
            for idx, date in enumerate(dates):
                if idx == 0:
                    record = [
                        serial, date, threshold, oh, ll, BR, daily_flight_hours, "Неактивно", 0, 0
                    ]
                else:
                    record = [
                        serial, date, threshold, oh, ll, BR, daily_flight_hours, None, None, None
                    ]
                data_to_insert.append(record)

        # Вставка данных
        client.execute(insert_query, data_to_insert, types_check=True)
        logger.info(f"Успешно вставлено {len(data_to_insert)} записей в OlapCube_VNV.")

    except Exception as e:
        logger.error(f"Ошибка при вставке данных в OlapCube_VNV: {e}", exc_info=True)

def main():
    logger.info("Начинаю вычисление aggregate_count.")
    aggregate_count = calculate_aggregate_count()

    if aggregate_count is not None:
        logger.info(f"aggregate_count рассчитан: {aggregate_count}")

        logger.info("Получаю значения из таблицы Agregat.")
        agregat_values = get_agregat_values()

        logger.info("Получаю daily_flight_hours из flight_hours.")
        daily_flight_hours = get_daily_flight_hours()

        logger.info("Начинаю вставку данных в OlapCube_VNV.")
        insert_into_olapcube_vnv_with_dates(aggregate_count, agregat_values, daily_flight_hours)
    else:
        logger.error("Не удалось рассчитать aggregate_count. Пропускаем вставку в OlapCube_VNV.")

if __name__ == "__main__":
    main()


2024-11-26 13:27:09,506 - INFO - Начинаю вычисление aggregate_count.
2024-11-26 13:27:09,514 - INFO - Сумма mi17_count: 840.0, сумма mi8t_count: 720.0
2024-11-26 13:27:09,521 - INFO - items_per_ac: 1.0, NR_Mi17: 6000.0
2024-11-26 13:27:09,526 - INFO - monthly_flight_hours: 48.0
2024-11-26 13:27:09,528 - INFO - Вычисленный aggregate_count: 206
2024-11-26 13:27:09,529 - INFO - aggregate_count рассчитан: 206
2024-11-26 13:27:09,531 - INFO - Получаю значения из таблицы Agregat.
2024-11-26 13:27:09,536 - INFO - Получены значения из таблицы Agregat.
2024-11-26 13:27:09,537 - INFO - Получаю daily_flight_hours из flight_hours.
2024-11-26 13:27:09,542 - INFO - Получено daily_flight_hours: 1.58
2024-11-26 13:27:09,543 - INFO - Начинаю вставку данных в OlapCube_VNV.
2024-11-26 13:27:09,555 - INFO - Получено 4001 уникальных дат.
2024-11-26 13:27:09,556 - INFO - Сгенерировано 206 серийных номеров.
2024-11-26 13:27:15,472 - INFO - Успешно вставлено 824206 записей в OlapCube_VNV.
